In [ ]:
import yfinance as yf
import pandas as pd  
from datetime import datetime, timedelta
import numpy as np  
from scipy.optimize import minimizepip
# Make imports work 

Define the list of tickers

In [5]:
tickers = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
#choose tickers for optimal portfolio gains. Considered industrial implications and past financial performance , stock valuation

Setting the date to 5 years prior

In [6]:
end_date = datetime.today()

NameError: name 'datetime' is not defined

In [ ]:
start_date = end_date - timedelta(days = 5*365)
print(start_date)

Create an empty dataframe  to collect closing prices and adjust for all variables

In [ ]:
adj_close_df = pd.DataFrame()

Downloading the close prices for each ticker

In [7]:
tickers = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
for ticker in tickers:
    data = yf.download(ticker, start= start_date, end = end_date)
    adj_close_df[ticker] = data['Adj Close']

NameError: name 'yf' is not defined

In [ ]:
print(adj_close_df)

Calculating Lognormal returns 

In [2]:
log_returns = np.log(adj_close_df/adj_close_df.shift(1))
log_returns = log_returns.dropna()

NameError: name 'np' is not defined

CALCULATING COVARIANCE MATRIX (Use 252 trading days)

In [ ]:
cov_matrix = log_returns.cov()*252
print(cov_matrix)
#displays covariance matrix with relative stock/index prices

STANDARD DEVIATION

In [ ]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

EXPECTED RETURN

In [ ]:
def expected_return(weights, log_return):
    return np.sum(log_returns.mean(*weights))*252
# average return calcuulated based on lognormal return and weights

CALCULATING SHARPE RATIO 

In [ ]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return(expected_return (weights, log_returns)- risk_free_rate) / standard_deviation(weights, cov_matrix)

CALCULATING RISK FREE RATE 
(using fred API to calculate average treasury bond rate
also can take standard assumption of 2%)

In [ ]:
from fredapi import Fred

fred = Fred(api_key= '2d60db8717ba3d033f961ac04cf4855e')
ten_year_treasury_rate = fred.get_series_latest_release('GS10' / 100)

#Set the risk free rate

risk_free_rate = ten_year_treasury_rate.iloc[-1]
print(risk_free_rate)

Function to minimize (Negative sharp ratio )

In [ ]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

Setting constraints: ensuring all portfolio weights are equal to one. Setting lower and upper bounds for the ratio of investments in multiple stocks. The constraint variable is set as a dictionary

In [ ]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) -1}
bounds = [(0, 0.5) for_in range(len(tickers))]

In [ ]:
initial_weights = np.array([1/len(tickers)]*len(tickers))

Optimizing weights to the maximum sharpe ratio 

In [ ]:
optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method= 'SLSQP', constraints=constraints, bounds=bounds)

# args are the arguements and constraints are used for optimal weights extraction

In [ ]:
optimal_weights = optimized_results.x

Data Visualisation of efficent frontiers/ weights

In [ ]:
print("optimal wights:")
for ticker, weight in zip(tickers, optimal_weights):
   print(f"{ticker}: {weight: .4f}") #understand print statement 


optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_vollatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected volatility: {optimal_portfolio_vollatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")


Final portfolio weightage plot

In [ ]:
import matplotlib.pyplot as plt

#creating a bar chart 

plt.figure(figsize=(10, 6)) #understnad figsize
plt.bar(tickers, optimal_weights)

#Add labels
plt.xlabel('Assets')
plt.ylabel('Optimal Weights')
plt.title('Optima portfolio Weights')

#Chart Display
plt.show()

MONTE CARLO SIMULATIONS

In [ ]:
#simulation imports 
from pandas_datareader import data as pdr


def get_data(stocks, start, end):
    StockData = pdr.get_data_yahoo(stocks, start, end)
    StockData = StockData['Close']
    returns = StockData.pct.change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix

stock_list = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)

weights = optimal_weights
print(weights)


In [ ]:
# Set MC simulations

mc_sims = 100
T = 100

Mean_matrix = np.full(shape=(T, len(weights)), fill_value = mean_returns) # setting matrix of return vectors
Mean_matrix = Mean_matrix.T #(transpose of the matrix)

Portfolio_sims = np.full(shape = (T, mc_sims), fill_value=0.0)

initial_value = 1000

for m in range(0, mc_sims):
    Z = np.random.normal(size=(T, len(weights)))
    L = np.linalg.cholesky(cov_matrix)
    Mc_return = Mean_matrix + np.inner(L, Z) #Using cholesky decomposition
    Portfolio_sims[: ,m] = np.cumprod(np.inner(weights, Mc_return.T)+1)*initial_value

    

MC visulaisation

In [ ]:
plt.plot(Portfolio_sims)
plt.ylabel('Value in $')
plt.xlabel('Days')
plt.title('MC simulation of a portfolio')
plt.show()